# Web Scrapping France Travail API Offres d'emploi

In [28]:
import requests
import os, sys
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time

### Importation des variables d'environnements

In [29]:
load_dotenv()

BASE_URL = "https://api.francetravail.io/partenaire/offresdemploi"
TOKEN_URL = "https://entreprise.francetravail.fr/connexion/oauth2/access_token?realm=/partenaire"
PUBLIC_KEY = os.environ.get("FT_PUBLIC_KEY")
PRIVATE_KEY = os.environ.get("FT_PRIVATE_KEY")

### Fonction de récupération du token

In [30]:
def get_token():
    try:
        res = requests.post(TOKEN_URL, data={
            "grant_type": "client_credentials",
            "client_id":PUBLIC_KEY,
            "client_secret":PRIVATE_KEY,
            "scope": "api_offresdemploiv2 o2dsoffre"
        }
    )
        return res
    except Exception as err :
        sys.exit(err)

### Fonction de récupération des requêtes en fonction des paramètres

In [31]:
def get_response_from_url(url,params):
    try:
        token = get_token()
        if token.status_code == 200:
            token = token.json().get('access_token')
            
        res = requests.get(
            url,
            headers={"Authorization": f"Bearer {token}"},
            params=params
        )
        return res
    except Exception as err :
        sys.exit(err)

### Récupération du premier élément de l'ENDPOINTS "Rechercher des offres" dans l'API "Offres d'emploi"

In [34]:
res = get_response_from_url(f"{BASE_URL}/v2/offres/search",params={'range':"0-0"})
print(f"taille : {len(res.json()['resultats'])}")
res.json()['resultats']

taille : 1


[{'id': '190PSZZ',
  'intitule': 'Regisseur / Regisseuse (H/F)',
  'description': "Pour une agence de location de jeux et d'animations pour adultes et enfants, vous travaillerez à la préparation des camions et installation sur sites. Vous procèderez ensuite au démontage, rangement et entretien du matériel.\n\nLe départ et le retour du matériel après la prestation se font  au Vigan (30120).\nL'entretien du matériel se fait toujours sur place, à l'entrepôt .\n\nTous les frais liés aux déplacements seront pris en charge : kms, restauration et hébergement. \nVenez rencontrer l'employeur le 30 avril. \nPour s'inscrire : https://mesevenementsemploi.francetravail.fr/mes-evenements-emploi/evenement/423999/recrutement-mes-scenes-de-star-le-vigan\n",
  'dateCreation': '2025-04-03T16:47:41.251Z',
  'dateActualisation': '2025-04-03T16:47:41.251Z',
  'lieuTravail': {'libelle': '30 - LE VIGAN',
   'latitude': 43.988139,
   'longitude': 3.613502,
   'codePostal': '30120',
   'commune': '30350'},
  'r

### Récupération de toutes les ranges possibles du paramètre range 

In [35]:
full_range_param = np.arange(0,3151,150)
full_range_param[-1] = full_range_param[-1]
full_range_param

array([   0,  150,  300,  450,  600,  750,  900, 1050, 1200, 1350, 1500,
       1650, 1800, 1950, 2100, 2250, 2400, 2550, 2700, 2850, 3000, 3150])

### Fonction de récupération de toutes les offres d'emploi

In [36]:
def get_all_data(range_call):
    res = np.array([])
    while len(range_call) > 1:
        d = range_call[0]
        p = range_call[1] - 1
        range_call = np.delete(range_call,0)

        print(f"{d}-{p}")
        res = np.append(
            res,
            get_response_from_url(f"{BASE_URL}/v2/offres/search",params={'range':f'{d}-{p}'})
            .json()['resultats']
        )
        time.sleep(0.10)
        

    return res


In [37]:
res = get_all_data(full_range_param)

0-149
150-299
300-449
450-599
600-749
750-899
900-1049
1050-1199
1200-1349
1350-1499
1500-1649
1650-1799
1800-1949
1950-2099
2100-2249
2250-2399
2400-2549
2550-2699
2700-2849
2850-2999
3000-3149


In [38]:
df = pd.DataFrame(res.tolist())

In [39]:
df.shape

(3150, 41)

In [43]:
df.head()

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,offresManqueCandidats,contexteTravail,formations,deplacementCode,deplacementLibelle,permis,langues,experienceCommentaire,complementExercice,conditionExercice
0,190PTBF,Formateur Services à la Personne (H/F),"L'ENTREPRISE :\n\nVIA Formation, entité du Gro...",2025-04-03T16:48:02.846Z,2025-04-03T16:48:03.420Z,"{'libelle': '37 - La Riche', 'latitude': 47.38...",K2111,Formateur / Formatrice,Formateur / Formatrice,"{'nom': 'VIA FORMATION', 'description': 'Via F...",...,False,{'horaires': ['temps partiel - 15H45 Travail e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,190PTBD,Audioprothésiste Franchisé (H/F),A la recherche d'une nouvelle opportunité prof...,2025-04-03T16:47:58.678Z,2025-04-03T16:47:59.170Z,{'libelle': 'Pays de la Loire'},J1401,Audioprothésiste,Audioprothésiste,{'entrepriseAdaptee': False},...,False,{},"[{'codeFormation': '43495', 'domaineLibelle': ...",1,Jamais,NaN,NaN,NaN,NaN,NaN
2,190PSZZ,Regisseur / Regisseuse (H/F),Pour une agence de location de jeux et d'anima...,2025-04-03T16:47:41.251Z,2025-04-03T16:47:41.251Z,"{'libelle': '30 - LE VIGAN', 'latitude': 43.98...",L1509,Régisseur / Régisseuse de production,Régisseur / Régisseuse de production,{'entrepriseAdaptee': False},...,False,{'horaires': ['35H Port et manipulation de cha...,NaN,3,Fréquents,"[{'libelle': 'B - Véhicule léger', 'exigence':...",NaN,NaN,NaN,NaN
3,190PSZV,Agent technique polyvalent en milieux rural (H...,Savoir travailler au sein d'une équipe réduite...,2025-04-03T16:47:22.489Z,2025-04-03T16:47:39.191Z,"{'libelle': '73 - saint paul sur YENNE', 'lat...",I1203,Agent / Agente d'entretien du bâtiment,Ouvrier(ère) polyvalent(e) d'entretien des bât...,{'nom': 'salon de l'emploi public territorial ...,...,False,"{'horaires': ['35H Autre'], 'conditionsExercic...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,190PSZT,Auxiliaire de vie (H/F),Nous recherchons un(e) Auxiliaire de Vie motiv...,2025-04-03T16:47:21.587Z,2025-04-03T16:47:22.280Z,"{'libelle': '65 - ARGELES GAZOST', 'latitude':...",K1311,Assistant / Assistante de vie aux familles,Auxiliaire de vie,"{'nom': 'VITALLIANCE', 'description': 'Service...",...,False,{'horaires': ['35H Travail en journée']},NaN,2,Ponctuels Zone départementale,"[{'libelle': 'B - Véhicule léger', 'exigence':...",NaN,NaN,NaN,NaN


### Export csv

In [41]:
df.to_csv('full_recherche_emploi_brut.csv')